# 제주도 핫플레이스 - 인스타그램 크롤링

In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [4]:
chromedriver = 'C:/Users/imkyu/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram 접속하여 로그인

In [5]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [6]:
with open('password.txt') as f:
    password = f.read()

In [7]:
email = 'llibeauty'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [8]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [9]:
# 팝업-알림(로그인정보 저장-안함)
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [10]:
# 팝업 - 알림(나중에 하기)
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도 맛집 으로 검색

In [11]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [12]:
driver.get(url)
time.sleep(5)   #시간 충분히 줘야 함(화면전환될떄까지기달)

### 3. 첫번째 게시글 열기

In [13]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 4. 게시글 정보 가져오기

In [14]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [15]:
# 1)본문내용 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'협찬환상적인 바다뷰를 지닌 숙성근고기 흑돼지맛집 💘💧064 - 711 - 3733💧제주시 애월읍 애월해안로 907💧매일 11:00 ~ 23:00(연중무휴)💧브레이크타임 NO💧런치세트 11:00 ~ 14:00#애월리식당#애월맛집 #애월흑돼지 #제주애월맛집 #제주도맛집#제주맛집#애월흑돼지맛집 #애월흑돼지 #제주흑돼지 #제주도흑돼지 #협재맛집 #제주공항근처맛집 #애월해안도로맛집#한림맛집 #애월읍맛집'

In [16]:
# 2)본문 내용에서 해시태그만 가져오기(정규표현식: '#[^\s#,\\]+')
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월리식당',
 '#애월맛집',
 '#애월흑돼지',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#애월흑돼지맛집',
 '#애월흑돼지',
 '#제주흑돼지',
 '#제주도흑돼지',
 '#협재맛집',
 '#제주공항근처맛집',
 '#애월해안도로맛집',
 '#한림맛집',
 '#애월읍맛집']

In [17]:
# 3)작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-17'

In [18]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [19]:
# 5) 위치정보 가져오기
try:
    place = soup.select_one('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [20]:
row = [content, date, like, place, tags]
row

['협찬환상적인 바다뷰를 지닌 숙성근고기 흑돼지맛집 💘💧064 - 711 - 3733💧제주시 애월읍 애월해안로 907💧매일 11:00 ~ 23:00(연중무휴)💧브레이크타임 NO💧런치세트 11:00 ~ 14:00#애월리식당#애월맛집 #애월흑돼지 #제주애월맛집 #제주도맛집#제주맛집#애월흑돼지맛집 #애월흑돼지 #제주흑돼지 #제주도흑돼지 #협재맛집 #제주공항근처맛집 #애월해안도로맛집#한림맛집 #애월읍맛집',
 '2021-08-17',
 0,
 '',
 ['#애월리식당',
  '#애월맛집',
  '#애월흑돼지',
  '#제주애월맛집',
  '#제주도맛집',
  '#제주맛집',
  '#애월흑돼지맛집',
  '#애월흑돼지',
  '#제주흑돼지',
  '#제주도흑돼지',
  '#협재맛집',
  '#제주공항근처맛집',
  '#애월해안도로맛집',
  '#한림맛집',
  '#애월읍맛집']]

### 5. 다음 게시글로 이동

In [21]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [22]:
# 6을 하기 위해 팝업 화면을 없앰(즉, X를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

### 6. 여러 개의 게시글 가져오기

In [23]:
from tqdm.notebook import tqdm

In [24]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0

    try:
        place = soup.select_one('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    return [content, tags, date, like, place]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [25]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [26]:
#200개 정도가 한계
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

  0%|          | 0/200 [00:00<?, ?it/s]

In [27]:
df = pd.DataFrame(results, columns=['content','tags', 'date','like','place'])
df.head()

,content,tags,date,like,place
0,협찬환상적인 바다뷰를 지닌 숙성근고기 흑돼지맛집 💘💧064 - 711 - 3733💧...,"[#애월리식당, #애월맛집, #애월흑돼지, #제주애월맛집, #제주도맛집, #제주맛집...",2021-08-17,0,
1,-배거픔에 잠안오는 야심한밤에 올려보는제주 오마카세정보🤪다녀온곳은 #제주하찌 였어요...,"[#제주하찌, #배고픈밤, #참자, #제주오마카세, #제주도오마카세, #서귀포맛집,...",2021-08-18,386,
2,42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐...,"[#신해바라기분식, #제주도맛집뜨끈한, #순두부찌개, #국밥맛집, #제주맛집, #제...",2021-08-18,"1,086",
3,<광고>#제주분식 https://jejubunsig.modoo.at/ 📌제주시 귀아...,"[#제주분식, #제주분식⠀간편하게, #모당모당, #돔베라면돔베김밥, #제주맛집, #...",2021-08-16,0,
4,"협찬제주애월맛집 ""랍스터빈""주소 : 제주시 애월읍 애월해안로 752전화 : 064-...","[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,...",2021-08-19,0,


In [30]:
df.tail()

,content,tags,date,like,place
174,날씨 굳😎,[],2021-08-19,35,
175,음악이 빵빵~ 제주펍 픽미 좋습니다#픽미펍클럽#제주도클럽#제주클럽#월정리클럽#제주도...,"[#픽미펍클럽, #제주도클럽, #제주클럽, #월정리클럽, #제주도술집, #월정리술집...",2021-08-19,1,
176,#통발오늘 제가 소개해드릴 맛집은 일식 전문점입니다앞에 바다도 훤히 다 보이고 분위...,[#통발오늘],2021-08-19,저 좋아요를 눌러보세,
177,(광고)정상 영업합니다제주 유일 해산물 모듬장 ♥애월제주다예약하고 꼭 드셔보세요~잊...,"[#애월맛집, #제주공항맛집, #제주공항근처맛집, #애월해안도로맛집, #협재맛집, ...",2021-08-19,367,
178,가족끼리 또 오자고해서 왔는데 역시나 마음에 드네요. 갈치구이에 고소한 맛 잊지못하...,"[#서귀포맛집, #색달식당중문본점, #서귀포통갈치조림, #제주맛집, #중문맛집, #...",2021-08-19,저 좋아요를 눌러보세,


In [28]:
df.to_csv('Data/제주맛집.csv', index=False)

In [29]:
driver.close()

- 4가지 CSV 파일의 중복을 제거한 후 통합 저장

In [31]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'data/{keyword}.csv')
    print(df.head())

                                             content  \
0  .#연동 #모니카옛날통닭간판부터 맛집포스 작렬하는 찐맛집이젠 인생치킨이라고 말하기도...   
1  <광고>섭지코지에서 한끼 맛있게 드시고 싶다면...특히 점심메뉴로 강력추천드리고요 ...   
2  .< 제주해물라면 맛집모음! > 일단저장@ 해물 아낌없이 주는 해물라면 맛집!바다뷰...   
3  협찬제주애월맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-...   
4  .한림 #문도지오름방목된 조랑말과 교감까지 할 수이따규~일몰 시간에 맞춰서 가면 멋...   

                                                tags        date   like place  
0  ['#연동', '#모니카옛날통닭간판부터', '#제주맛집', '#제주도맛집', '#제...  2021-08-19  1,113   NaN  
1  ['#섭지코지한끼🍱오픈', '#섭지코지맛집', '#섭지코지', '#섭지코지카페', ...  2021-08-18      0   NaN  
2  ['#제주맛집', '#제주도맛집', '#제주공항근처맛집', '#제주시맛집', '#애...  2021-08-18  1,008   NaN  
3  ['#애월맛집', '#애월카페', '#애월읍맛집', '#제주애월맛집', '#제주도맛...  2021-08-18      0   NaN  
4  ['#문도지오름방목된', '#제주맛집', '#제주도맛집', '#제주공항근처맛집', ...  2021-08-17  1,304   NaN  
                                             content  \
0  협찬환상적인 바다뷰를 지닌 숙성근고기 흑돼지맛집 💘💧064 - 711 - 3733💧...   
1  -배거픔에 잠안오는 야심한밤에 올려보는제주 오마카세정보🤪다녀온곳은 #제주하찌 였어요...   
2  42년 전통 도민 맛집